Extract a sample of 10.000 texts, annotated as News, from the following MaCoCu/CLASSLA-web corpora: Croatian, Slovenian, Catalan, Greek

I took the CLASSLA/MaCoCu corpora from `http://hdl.handle.net/11356/1969` that I previously prepared as JSONL files for the automatic genre annotation. The preparation of texts included the following additional post-processing steps:
- truncating texts to the first 512 words of each text (because the max_sequence_length that model can take is 512 tokens)
- removal of all texts that are in multiple languages - only texts that are only in the target language are kept

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Define paths to the MaCoCu-Genre datasets - an example
dataset_paths = {
	"el": "path_example"
}

In [ ]:
# Open jsonl file

lang = "el"

df = pd.read_json(dataset_paths[lang], orient="records", lines=True)

display(df.head())

In [6]:
# Take only texts, labeled as "News"

df = df[df["genre"] == "News"]

df.shape

(5301458, 7)

In [7]:
# Find texts with "multiple texts" inside based on whether there is more than one ellipsis
multiple_list = []

for text in df["text"].to_list():
    if text.count("...") > 1:
        multiple_list.append("yes")
    else:
        multiple_list.append("no")

df["multitext"] = multiple_list

df.head(2)

,document_id,text,genre,logit,lang_distr,lang,text_length,multitext
1,macocu.el.4,Πιο σοβαροί αποδεικνύονται οι τραυματισμοί των...,News,"[-1.4598587751, -0.6933389306000001, 7.8427786...","[('el', 1.0)]",el,NaN,no
2,macocu.el.5,Πρωτοβουλία του ΥφΑΑΤ Σίμου Κεδίκογλου για την...,News,"[-1.4910259247, -0.9129921794, 7.8686022758, -...","[('el', 1.0)]",el,NaN,no


Number of texts annotated as "News" that were removed due to having more than one "...":
- HR: 196,256 (8.4%)
- SL: 103,589 (9.2%)
- CA: 96,930 (5.6%)
- EL: 739,944 (14.0%)

In [10]:
# Remove from the sample texts that have more than one "..."
df = df[df["multitext"] == "no"]

df.shape


(4561514, 8)

In [ ]:
# Now slimmer down the sample to 10.000 instances
big_sample = df.sample(n=10000)

big_sample.shape

(10000, 8)

In [17]:
# Drop multitext column
big_sample.drop(columns=['multitext'], inplace=True)

In [19]:
# Change <p> signs to actual new lines
big_sample["text"] = big_sample["text"].str.replace("<p>", "\n\n")

with pd.option_context('display.max_colwidth', None):
	display(big_sample.head())


,document_id,text,genre
3973259,macocu.el.6373650,"Ομιλία Δημάρχου Πάρου για το Λιμενικό Σύστημα του νησιού Κύριο \n\nΗ ομιλία Δημάρχου Πάρου Μάρκου Ι. Κωβαίου, στα πλαίσια του επιστημονικού συνεδρίου με τίτλο ""Φορείς διαχείρισης και εποπτείας λιμένων Ελλάδος, υπάρχουσα κατάσταση, προτάσεις βελτίωσης"" που συνδιοργανώθηκε από το Πανεπιστήμιο Δυτικής Αττικής και τον Δήμο Πάρου, υπό την αιγίδα του Υπουργείου Ναυτιλίας και Νησιωτικής Πολιτικής, στην Πάρο 21 &amp; 22 Σεπτεμβρίου 2018, στην αίθουσα εκδηλώσεων του 1ου ΕΠΑΛ Πάρου, στο Λιαροκόπι Παροικιάς. \n\nΑπευθυνόμενος στον Παριανό λαό και τη κατασκευή του Νέου Εμπορικού Λιμένα, ο Δήμαρχος Πάρου επισήμανε: \n\n",News
3449533,macocu.el.5513105,"Βαραίνουν το κόστος οι ανατιμήσεις, ορατά τα 60 λεπτά στο βαμβάκι \n\nΩς κορυφή της αγοράς με τα σημερινά επίπεδα, προκύπτουν τα 93 σεντς, με την αγορά να επιχειρεί προ ολίγων ημερών την κατάκτηση των 90 σεντς, προτού μια γενικευμένη πτώση των χρηματαγορών ανακόψει την ορμή στην αγορά βάμβακος. Ακόμα και τότε όμως, η χρηματιστηριακή τιμή ανέκαμψε γρήγορα επιστρέφοντας στα 88 σεντς ανά λίμπρα, επίπεδο κοντά στα υψηλά εξαμήνου για τα συμβόλαια ∆εκεμβρίου '21. \n\nΣτην εγχώρια αγορά, ήδη γίνονται προπωλήσεις πάνω από τα 91 σεντς με την αγορά να πληρώνει στα εκκοκκιστήρια πριμ 3 έως και 4 σεντς επί της χρηματιστηριακής βάσης για τα λευκά βαμβάκια. Την ίδια στιγμή, οι προαγορές από τα εκκοκκιστήρια επέστρεψαν σε τιμές στα 55 με 55,5 λεπτά το κιλό για το σύσπορο, αφού πέραν της ανοδικής τάσης στη διεθνή τιμή, η ισοτιμία ευρώ/ δολάριο υποχώρησε από τα 1,21 ευρώ και έχει σταθεροποιηθεί εδώ και μερικές εβδομάδες στα 1,18 με 1,17 ευρώ, βελτιώνοντας άρα τις τιμές για το ελληνικό βαμβάκι. \n\nΉδη αρκετοί οργανωμένοι παραγωγοί στην Ελλάδα έχουν προβεί στην προπώληση περίπου του 50% της αναμενόμενης παραγωγής τους, με μπούσουλα τα 250 κιλά ανά στρέμμα, αναμένοντας τα χειμερινά φιξαρίσματα για το υπόλοιπο 50% της παραγωγής τους. Εν προκειμένω, φαίνεται πως η αγορά θα μπορούσε να διαμορφώσει υψηλότερα επίπεδα από τα σημερινά, παρά το γεγονός ότι ιστορικά, κατά τους φθινοπωρινούς μήνες οι τιμές υποχωρούν. Σε κάθε περίπτωση, η διασπορά του ρίσκου με προπωλήσεις καθ' όλη τη διάρκεια της καλλιεργητικής περιόδου, έναντι των φιξαρισμάτων του ∆εκεμβρίου, στέκει ως ένα αξιόλογο εργαλείο στα χέρια των παραγωγών, που φέτος για πρώτη φορά δοκιμάζεται στην εγχώρια αγορά. Εν τω μεταξύ, το φαινόμενο της ""Ψυχρής Λίμνης"" την περασμένη εβδομάδα, ακούγεται ότι θα κόψει κάποιες λίγες ποσότητες από την αναμενόμενη παραγωγή της χώρας, μιας και τουλάχιστον 5.000 στρέμματα βούλιαξαν σε περιοχές των Φαρσάλων και του Παλαμά. Υπενθυμίζεται ότι οι εκτάσεις της χώρας είναι μειωμένες κατά τουλάχιστον 15% φέτος. \n\nΕίναι γεγονός, πάντως, πως η φετινή συγκυρία διαφέρει από τον γενικό κανόνα. Εξερχόμενη από την πανδημία, η διεθνής αγορά βάμβακος έχοντας μέσα στο 2020 σημειώσει τιμές ακόμα και κάτω από τα 50 σεντς ανά λίμπρα, δείχνει πιο ενισχυμένη από ποτέ άλλοτε σε βάθος τετραετίας. Η ζήτηση παραμένει υψηλή, κάτι που αποτυπώνουν με συνέπεια οι εκθέσεις προσφοράς-ζήτησης του αμερικανικού υπουργείου Γεωργίας τους τελευταίους μήνες, όσο η φυτική ίνα φαίνεται πως επικρατεί όλο και με μεγαλύτερη αυτοπεποίθηση έναντι της συνθετικής. \n\nΣε ό,τι αφορά τα δεδομένα της παραγωγής και της κατανάλωσης, η έκθεση Ιουλίου του USDA περιόρισε στα 87,7 εκατ. δεμάτια τα παγκόσμια αποθέματα, συγκριτικά με τις εκτιμήσεις του περασμένου μήνα, που ανέμεναν ότι στο τέλος της τρέχουσας περιόδου θα έχουν περισσέψει περίπου 89,3 εκατ. δεμάτια. Ταυτόχρονα αυξημένη είναι και η παγκόσμια κατανάλωση, κατά 700.000 δεμάτια, στις 123,2 εκατ. μπάλες, με την αναμενόμενη παραγωγή για το 2021-2022 να αποτιμάται στα 119,4 εκατ. δεμάτια, κάτι που δεν έχει πείσει την αγορά, η οποία θεωρεί πως το συνολικό τονάζ θα είναι μικρότερο από τις προβλέψεις του USDA. \n\nΑδειάζουν οι αποθήκες στο Νότιο ημισφαίριο όσο η παραγωγή υπολείπεται των προσδοκιών \n\nΤα χαμηλότερα αποθέματα της τελευταίας τριετίας αναφέρει ο κλάδος βάμβακος 

In [ ]:
# Extract a sample on which we will first run the automatic prediction and mark it
prediction_sample = big_sample.sample(n=5250)

print(prediction_sample.shape)

prediction_sample.head(2)

(5000, 3)


,document_id,text,genre
7634994,macocu.el.12233021,Η Yahoo και η AOL για πούλημα έναντι $5 δισεκα...,News
5518538,macocu.el.8848752,Απάντηση σε ερώτηση για ΚΕΠΑ και ασφάλιση ΑΜΕΑ...,News


In [21]:
# Extract ids to mark in the big sample which text is included in the prediction sample
pred_sample_ids = prediction_sample["document_id"].to_list()

big_sample["prediction_sample"] = np.where((big_sample["document_id"].isin(pred_sample_ids)), "yes", "no" )

big_sample.head(10)

,document_id,text,genre,prediction_sample
3973259,macocu.el.6373650,Ομιλία Δημάρχου Πάρου για το Λιμενικό Σύστημα ...,News,no
3449533,macocu.el.5513105,"Βαραίνουν το κόστος οι ανατιμήσεις, ορατά τα 6...",News,no
4932053,macocu.el.7914732,Φόρμα αναζήτησης \n\nΜετά από 48ωρο κρεσέντο α...,News,no
7937082,macocu.el.12751875,Στο πλαίσιο της αναμέτρησης της Γερμανίας με τ...,News,no
125493,macocu.el.183717,ΜΕ 5 ΑΘΛΗΤΕΣ ΣΤΗΝ ΣΤΙΣ ΠΡΟΕΘΝΙΚΕΣ ΟΜΑΔΕΣ OPTIM...,News,no
10277535,macocu.el.16270178,Ο Δήμος Ναυπακτίας για την επιστολή της Ομοσπο...,News,yes
9894302,macocu.el.15610009,Προκηρύξεις \n\nΠΡΟΟΙΜΙΟ \n\n8.30 - 9.30: Λειτ...,News,no
5921875,macocu.el.9530596,Σελίδες \n\nΔεύτερη ευκαιρία για τους Δημοτικο...,News,no
2468336,macocu.el.3957210,2ο Πανελλήνιο Φεστιβάλ Κόμικς \n\nΤο 2ο Φεστιβ...,News,no
4389695,macocu.el.7061159,Ποδαρικό στο πρωτάθλημα με ντέρμπι \n\nΜετά απ...,News,no


In [ ]:
# Save the dataset
big_sample.to_json("datasets/MaCoCu-sample/MaCoCu-training-sample-{}.jsonl".format(lang), orient="records", lines=True)